## First Layer

In [158]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

import datetime
import time
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np
from datetime import date
date_today = date.today()

options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_argument("--disable-extensions")
driver = webdriver.Chrome(options=options, executable_path=r'/Users/ricardolu/Desktop/Design lab project/chromedriver')
driver.set_window_position(0, 0)
driver.set_window_size(1024, 768)
soups = []

for i in range(1,11):
    driver.get("https://www.levi.com/US/en_US/clothing/men/c/levi_clothing_men?page="+str(i))
    time.sleep(5)
    #X_button = driver.find_element_by_class_name('modal__close')
    #X_button.click()
    soup = BeautifulSoup(driver.page_source,'html.parser')
    soups.append(soup)
    

#print(soup.prettify())

In [159]:
product_name = []
sale_price = []
original_price = []
product_url = []
image_url = []
brand = []
product_type = []
timestamp = []

for soup in soups:
    for i in soup.find_all('div',{'class':'product-cell'}):
        product_name.append(i.find('div',{'class':'product-name'}).text)
        if i.find('div',{'class':'product-price'}) != 0:
            sale_price.append(i.find('div',{'class':'product-price'}).text.strip())
        product_url.append('https://www.levi.com'+str(i.find('a',{'class':'product-link lsco-col-xs-offset-1 lsco-col-lg-offset-0 lsco-col-xs-offset-right-1 lsco-col-lg-offset-right-0'}).get('href')))
        if (i.find('div',{'class':'image-container'}).find('img',{'class':'processed-image'})) !=None:
            image_url.append(i.find('div',{'class':'image-container'}).find('img',{'class':'processed-image'}).get('data-src'))
        else:
            image_url.append(None)
        brand.append("levis")
        product_type.append("Men's Clothing")
        timestamp.append(date_today)
        

In [160]:
for i, v in enumerate(sale_price):
    if "Sale Price" in v:
        sale_price[i] = v.replace("Sale Price", "").split("Original Price")[0]
        original_price.append(v.replace("Sale Price", "").split("Original Price")[1].replace("Temporary",""))
    else:
        original_price.append(None)  

In [161]:
import pandas as pd

column_names = ["product_name", "original_price", "sale_price",'product_link', 'image_link']
df = pd.DataFrame(columns = column_names)


df['product_name'] = product_name
df['original_price'] = original_price
df['sale_price'] = sale_price
df['image_link'] = image_url
df['product_link'] = product_url

In [162]:
df

,product_name,original_price,sale_price,product_link,image_link
0,511™ Slim Fit Men's Jeans,None,$79.50,www.levi.com/US/en_US/clothing/men/jeans/511-s...,https://lsco.scene7.com/is/image/lsco/04511150...
1,Reusable Reversible Printed Face Mask (3 Pack),None,$15.00,www.levi.com/US/en_US/accessories/bandanas/reu...,https://lsco.scene7.com/is/image/lsco/levis/cl...
2,510™ Skinny Fit Men's Jeans,$69.50,Temporary $49.99 - $69.50,www.levi.com/US/en_US/apparel/clothing/bottoms...,https://lsco.scene7.com/is/image/lsco/05510069...
3,569™ Loose Straight Fit Men's Jeans,$59.50,Temporary $39.99,www.levi.com/US/en_US/apparel/clothing/bottoms...,https://lsco.scene7.com/is/image/lsco/00569024...
4,541™ Athletic Taper Levi’s® Flex Men's Jeans,$69.50,Temporary $49.99,www.levi.com/US/en_US/clothing/men/jeans/541-a...,https://lsco.scene7.com/is/image/lsco/18181021...
...,...,...,...,...,...
355,Levi's® x Peanuts Pocket Tee Shirt,$25.00,$16.98,www.levi.com/US/en_US/clothing/men/shirts/levi...,https://lsco.scene7.com/is/image/lsco/34310001...
356,511™ Slim Fit Levi’s® Flex Men's Jeans,$89.50,$67.98,www.levi.com/US/en_US/apparel/clothing/bottoms...,https://lsco.scene7.com/is/image/lsco/levis/cl...
357,LEGO Group x Levi's® 501® '93 Straight Fit Me...,$148.00,$79.98,www.levi.com/US/en_US/clothing/men/jeans/lego-...,https://lsco.scene7.com/is/image/lsco/79830008...
358,Levi's® x Peanuts Football Tee Shirt,$40.00,$21.98,www.levi.com/US/en_US/clothing/men/shirts/levi...,https://lsco.scene7.com/is/image/lsco/23895000...


## Second Layer

In [172]:
product_description = []
material = []
picture_name = []
color = []
counter = 1

for link in product_url:
    driver.get(link)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    
    item_color = []
    if soup.find('div',{'class':'swatches'}) != None:
        for color_i in soup.find('div',{'class':'swatches'}).find_all('li'):
            item_color.append(color_i.get("aria-label"))
        color.append(item_color)
    else:
        color.append(None)
    
    if len(soup.find_all('div',{'class':'contentContainer'})) >= 2:
        item_material = soup.find_all('div',{'class':'contentContainer'})[1].find("li").text.strip()
        material.append(item_material)
    else:
        material.append(None)
    
    if len(soup.find_all('div',{'class':'contentContainer'})) >= 1:
        if (soup.find_all('div',{'class':'contentContainer'})[0].find("li")) != None:
            product_description.append(soup.find_all('div',{'class':'contentContainer'})[0].find("li").text.strip())
        else:
            product_description.append(None)
    else:
        product_description.append(None)

    picture_name.append('levis-men_' + str(counter) + '.jpg') # change variable
    counter += 1
driver.close()

NameError: name 'drive' is not defined

In [186]:
for i,v in enumerate(color):
    if v != None:
        for u,j in enumerate(v):
            if j != None:
                color[i][u] = j.replace(" Medium Wash","").replace("- ","").replace(" Dark Wash","").replace(" Light Wash","").replace(" 3D Washed","")
        
for i,v in enumerate(sale_price):
    if v != None:
        sale_price[i] = v.replace("Temporary","")

In [190]:
column_names = ["brand", "product_type", "product_name", 'product_description', "price", 
                "sales_price", "color","material", "product_url", "picture_url", "timestamp", 'picture_name']
second_pass = pd.DataFrame(columns = column_names)
second_pass.brand = brand
second_pass.product_type = product_type
second_pass.product_name = product_name
second_pass.product_description = product_description
second_pass.price = original_price
second_pass.sales_price = sale_price
second_pass.color = color
second_pass.material = material
second_pass.product_url = product_url
second_pass.picture_url = image_url
second_pass.timestamp = timestamp
second_pass.picture_name = picture_name
second_pass.head()

,brand,product_type,product_name,product_description,price,sales_price,color,material,product_url,picture_url,timestamp,picture_name
0,levis,Men's Clothing,511™ Slim Fit Men's Jeans,A modern slim with room to move,None,$79.50,"[Nightshine Black, Chain Rinse, Baltic Navy Su...",Slim through the seat and thigh,https://www.levi.com/US/en_US/clothing/men/jea...,https://lsco.scene7.com/is/image/lsco/04511150...,2020-12-09,levis-men_1.jpg
1,levis,Men's Clothing,Reusable Reversible Printed Face Mask (3 Pack),Essential 3-pack of reusable face masks,None,$15.00,[Multi-Color Printed Paisley],Secure fit,https://www.levi.com/US/en_US/accessories/band...,https://lsco.scene7.com/is/image/lsco/levis/cl...,2020-12-09,levis-men_2.jpg
2,levis,Men's Clothing,510™ Skinny Fit Men's Jeans,The ultimate skinny fit for men,$69.50,$49.99 - $69.50,"[Dark Hollow, Reznor, Sinaloa ]",Skinny through the seat and thigh,https://www.levi.com/US/en_US/apparel/clothing...,https://lsco.scene7.com/is/image/lsco/05510069...,2020-12-09,levis-men_3.jpg
3,levis,Men's Clothing,569™ Loose Straight Fit Men's Jeans,"A modern, laid-back fit.",$59.50,$39.99,"[Kale, Cleaner, Crosstown, Black, Vintage Ligh...",Extra room in seat and thigh,https://www.levi.com/US/en_US/apparel/clothing...,https://lsco.scene7.com/is/image/lsco/00569024...,2020-12-09,levis-men_4.jpg
4,levis,Men's Clothing,541™ Athletic Taper Levi’s® Flex Men's Jeans,Perfect fit for athletic builds,$69.50,$49.99,"[Husker, Cleaner, Begonia , Native Cali]",Extra room through the seat and thigh,https://www.levi.com/US/en_US/clothing/men/jea...,https://lsco.scene7.com/is/image/lsco/18181021...,2020-12-09,levis-men_5.jpg


In [191]:
second_pass.to_csv("levis-men.csv", index=False, encoding = 'utf-8-sig')

In [175]:
import os
import requests as req

new_pd = pd.read_csv('levis-men.csv', encoding='utf-8-sig')

os.mkdir('levis-men')
i = 1
for index, link in enumerate(new_pd['picture_url']):
    img_data = req.get(link).content
    with open('levis-men/'+ "levis-men_"+str(index+1)+'.jpg','wb+') as f:
        f.write(img_data)
    i+=1

In [176]:
len(product_description)

360